In [1]:
import re
import requests

import pandas as pd

from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

from tqdm.auto import tqdm

### DATA SOURCE: https://tr.wikipedia.org/wiki/%C4%B0stanbul%27daki_iskeleler_listesi

In [2]:
url="https://tr.wikipedia.org/wiki/%C4%B0stanbul%27daki_iskeleler_listesi"

r = requests.get(url)
print(r)
soup=BeautifulSoup(r.text, "lxml")

<Response [200]>


In [3]:
rows = (
    soup.find(name="table", attrs={"class" : "wikitable sortable"})
    .find_all("tr")[3:]
)

In [4]:
ferries=[]
for row in rows:
    try:
        parsed_text = row.find_all("td")[0].a["title"]
        extracted_text = re.search(r'^(.+ İskelesi)', parsed_text).group(1)

        ferries.append(extracted_text)
        
    except (TypeError, IndexError):
        pass

In [5]:
ferries

['Anadoluhisarı İskelesi',
 'Anadolukavağı İskelesi',
 'Arnavutköy İskelesi',
 'Aşiyan İskelesi',
 'Avcılar İskelesi',
 'Ayvansaray İskelesi',
 'Bakırköy İskelesi',
 'Balat İskelesi',
 'Barbaros Hayrettin Paşa İskelesi',
 'Bebek İskelesi',
 'Beşiktaş İskelesi',
 'Beykoz İskelesi',
 'Beylerbeyi İskelesi',
 'Bostancı İskelesi',
 'Burgazada İskelesi',
 'Büyükada İskelesi',
 'Büyükdere İskelesi',
 'Çengelköy İskelesi',
 'Çubuklu İskelesi',
 'Eminönü İskelesi',
 'Emirgan İskelesi',
 'Eyüpsultan İskelesi',
 'Fener İskelesi',
 'Hasköy İskelesi',
 'Heybeliada İskelesi',
 'İstinye İskelesi',
 'Kabataş İskelesi',
 'Kadıköy İskelesi',
 'Kandilli İskelesi',
 'Kanlıca İskelesi',
 'Karaköy İskelesi',
 'Kasımpaşa İskelesi',
 'Kınalıada İskelesi',
 'Kuruçeşme İskelesi',
 'Kuzguncuk İskelesi',
 'Küçüksu İskelesi',
 'Maltepe İskelesi',
 'Ortaköy İskelesi',
 'Paşabahçe İskelesi',
 'Poyraz İskelesi',
 'Rumelikavağı İskelesi',
 'Sarıyer İskelesi',
 'Sedefadası İskelesi',
 'Sütlüce İskelesi',
 'Üsküdar İske

In [6]:
# These ferries did not return any results, so creating a seperate list to handle them
exception_ferries = [
    "Anadolukavağı İskelesi", "Aşiyan İskelesi", "Büyükdere İskelesi", "Hasköy İskelesi", "Kasımpaşa İskelesi", "Kuruçeşme İskelesi",
    "Poyraz İskelesi",  "Rumelikavağı İskelesi", "Sedefadası İskelesi", "Yenikapı İskelesi"
]

In [7]:
def transform_text(name):

    if "Büyükdere" in name:
        return name.replace("Büyükdere", "Büyük dere")
    
    elif "Yenikapı" in name:
        return name.replace("Yenikapı İskelesi", "İDO Yenikapı Terminalı")
    
    else:
        cleaned_name = (
            name.replace("kavağı", " kavağı")
            .replace("İskelesi", "ferry terminal")
        )

        return cleaned_name

In [8]:
cleaned_ferries = [transform_text(name=name) if name in exception_ferries else name for name in ferries]

In [9]:
geolocator = Nominatim(user_agent="istanbul_ferries")

In [10]:
ferry_info=[]
for ferry in tqdm(cleaned_ferries):
    
    location = geolocator.geocode(ferry)

    if location:
        ferry_info.append({
            "ferry_terminal" : ferry,
            "latitude" : location.latitude,
            "longitude" : location.longitude,
            "ferry_terminal_address" : location.address
        })


    else:
        print(ferry)

  0%|          | 0/49 [00:00<?, ?it/s]

Kuruçeşme ferry terminal
Poyraz ferry terminal
Sedefadası ferry terminal


#### Kuruçeşme is permenantly closed
#### No information on Poyraz
#### Sedef Adasi is a small island, therefore excluding it from the list.

In [11]:
df = pd.DataFrame(ferry_info)

In [13]:
df["ferry_terminal"] = df["ferry_terminal"].replace("ferry terminal|Terminalı", "İskelesi", regex=True)

In [14]:
df

,ferry_terminal,latitude,longitude,ferry_terminal_address
0,Anadoluhisarı İskelesi,41.083232,29.066641,"Anadolu Hisarı İskelesi, Körfez Caddesi, Anado..."
1,Anadolu kavağı İskelesi,41.173428,29.087870,"Anadolu Kavağı, Anadolu Kavağı Mahallesi, Beyk..."
2,Arnavutköy İskelesi,41.067041,29.044139,"Arnavutköy İskelesi İsbike İstasyonu, Bebek Ar..."
3,Aşiyan İskelesi,41.081140,29.054862,"Aşiyan, Yahya Kemal Caddesi, Rumelihisarı Maha..."
4,Avcılar İskelesi,40.971084,28.717951,"Avcılar İDO İskelesi, Dr. Sadık Ahmet Caddesi,..."
5,Ayvansaray İskelesi,41.034817,28.949212,"Balat iskelesi, Ayvansaray Caddesi, Fener, Bal..."
6,Bakırköy İskelesi,40.973028,28.877826,"İDO Bakırköy İskelesi, Kennedy Caddesi, Zeytin..."
7,Balat İskelesi,41.034817,28.949212,"Balat iskelesi, Ayvansaray Caddesi, Fener, Bal..."
8,Barbaros Hayrettin Paşa İskelesi,41.040036,29.005587,"Beşiktaş - Kadıköy İskelesi, Hayrettin İskeles..."
9,Bebek İskelesi,41.067041,29.044139,"Arnavutköy İskelesi İsbike İstasyonu, Bebek Ar..."


In [15]:
df.to_csv("tr_ist_ferry_terminals.csv", index=False)